<a href="https://colab.research.google.com/github/uhdang/mlim_group_3_final_assignment/blob/moving_window/project/train_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data Using Google Colab
Run if using Google Colab

In [2]:
import pandas as pd
import os
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/My Drive')

from dataloader import Dataloader
# Load Data
path = "/content/drive/MyDrive/My Drive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data Locally
Run if using notebook locally

In [1]:
import pandas as pd
import os
from dataloader import Dataloader

# Load Data
path = os.getcwd() + "/../data/"

In [3]:
# load data which creates baskets and coupons
data = Dataloader(path)

In [4]:
# Create Categories before CV as they don't change and consume time otherwiser
data.create_category_table(2000)

In [ ]:
# weeks = [87, 88, 89]
weeks = [89]
# shopper_list = list(range(2000))
# shopper_chunks = [shopper_list[i:i + 100] for i in range(0, len(shopper_list), 100)]
shopper_chunks = [list(range(100))]

for week in weeks:
    for shopper in shopper_chunks:
        # train-test-split
        data.train_test_split(week, shopper)

        # data add categories 
        data.add_categories()

        # create features
        data.create_feature_dict()

        # combine everything
        X_train, y_train, X_test, y_test = data.make_featured_data()

In [6]:
X_train

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,ratio_of_reordered_products,ratio_of_reordered_categories,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order,count_of_product_order_last_3_weeks,count_of_category_order_last_3_weeks,count_of_product_order_last_5_weeks,count_of_category_order_last_5_weeks,count_of_product_order_last_15_weeks,count_of_category_order_last_15_weeks,count_of_product_order_last_30_weeks,count_of_category_order_last_30_weeks
0,0,0,688.0,15,0.0,No,688,0,0.0,8,1.0,Yes,0.722222,0.920000,587.203947,8.539326,54,25,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,688.0,15,0.0,No,688,6,1.0,28,1.0,Yes,0.830986,1.000000,584.065350,7.393258,71,23,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,688.0,15,0.0,No,688,0,0.0,53,1.0,No,0.700000,0.909091,571.918919,5.820225,60,22,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,688.0,15,0.0,No,688,0,0.0,37,1.0,No,0.714286,0.958333,561.846361,8.337079,91,24,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,688.0,15,0.0,No,688,0,0.0,8,1.0,No,0.764706,1.000000,583.750455,6.168539,34,20,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224995,95,249,499.0,13,0.0,No,499,1,0.0,31,1.0,No,0.682353,1.000000,601.442997,6.898876,85,25,58,3,0.0,1.0,0.0,1.0,0.0,5.0,0.0,11.0
2224996,96,249,499.0,13,0.0,No,499,0,0.0,16,1.0,No,0.687500,1.000000,565.047923,7.033708,80,25,89,4,0.0,0.0,0.0,1.0,0.0,3.0,0.0,6.0
2224997,97,249,499.0,13,0.0,No,499,1,0.0,1,0.0,No,0.620253,0.960000,592.612587,8.033708,79,25,8,8,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
2224998,98,249,499.0,13,0.0,No,499,23,1.0,23,1.0,No,0.782609,0.960000,574.548578,9.483146,69,25,4,4,1.0,0.0,2.0,1.0,4.0,3.0,8.0,7.0


In [7]:
X_test

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,ratio_of_reordered_products,ratio_of_reordered_categories,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order,count_of_product_order_last_3_weeks,count_of_category_order_last_3_weeks,count_of_product_order_last_5_weeks,count_of_category_order_last_5_weeks,count_of_product_order_last_15_weeks,count_of_category_order_last_15_weeks,count_of_product_order_last_30_weeks,count_of_category_order_last_30_weeks
0,0,0,688.0,15,0.0,No,688,0,0.0,8,1.0,No,0.722222,0.92,587.203947,8.539326,54,25,90,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,0,1,560.0,15,0.0,No,560,0,0.0,8,1.0,No,0.722222,0.92,587.203947,8.539326,54,25,90,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,0,2,773.0,15,0.0,No,773,0,0.0,8,1.0,No,0.722222,0.92,587.203947,8.539326,54,25,90,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,0,3,722.0,15,0.0,No,722,0,0.0,8,1.0,No,0.722222,0.92,587.203947,8.539326,54,25,90,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0,4,620.0,15,0.0,No,620,8,1.0,8,1.0,No,0.722222,0.92,587.203947,8.539326,54,25,19,19,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,99,245,549.0,13,0.0,No,549,0,0.0,42,1.0,No,0.763158,0.96,618.717752,8.797753,76,25,90,12,0.0,0.0,0.0,0.0,0.0,4.0,0.0,10.0
24996,99,246,702.0,13,0.0,No,702,41,1.0,42,1.0,No,0.763158,0.96,618.717752,8.797753,76,25,12,12,0.0,0.0,0.0,0.0,4.0,4.0,10.0,10.0
24997,99,247,670.0,13,0.0,No,670,0,0.0,42,1.0,No,0.763158,0.96,618.717752,8.797753,76,25,90,12,0.0,0.0,0.0,0.0,0.0,4.0,0.0,10.0
24998,99,248,490.0,13,0.0,No,490,0,0.0,42,1.0,No,0.763158,0.96,618.717752,8.797753,76,25,90,12,0.0,0.0,0.0,0.0,0.0,4.0,0.0,10.0


In [9]:
X_test.tail(20)

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,reordered_category,coupon_in_same_category,ratio_of_reordered_products,ratio_of_reordered_categories,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order,count_of_product_order_last_3_weeks,count_of_category_order_last_3_weeks,count_of_product_order_last_5_weeks,count_of_category_order_last_5_weeks,count_of_product_order_last_15_weeks,count_of_category_order_last_15_weeks,count_of_product_order_last_30_weeks,count_of_category_order_last_30_weeks
24980,98,240,528.0,13,0.0,No,528,0,0.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,90,5,0.0,1.0,0.0,2.0,0.0,4.0,0.0,8.0
24981,98,241,699.0,13,0.0,No,699,0,0.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,90,5,0.0,1.0,0.0,2.0,0.0,4.0,0.0,8.0
24982,98,242,511.0,13,0.0,No,511,0,0.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,90,5,0.0,1.0,0.0,2.0,0.0,4.0,0.0,8.0
24983,98,243,490.0,13,0.0,No,490,0,0.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,90,5,0.0,1.0,0.0,2.0,0.0,4.0,0.0,8.0
24984,98,244,694.0,13,0.0,No,694,0,0.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,90,5,0.0,1.0,0.0,2.0,0.0,4.0,0.0,8.0
24985,98,245,549.0,13,0.0,No,549,0,0.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,90,5,0.0,1.0,0.0,2.0,0.0,4.0,0.0,8.0
24986,98,246,702.0,13,0.0,No,702,0,0.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,90,5,0.0,1.0,0.0,2.0,0.0,4.0,0.0,8.0
24987,98,247,670.0,13,0.0,No,670,0,0.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,90,5,0.0,1.0,0.0,2.0,0.0,4.0,0.0,8.0
24988,98,248,490.0,13,0.0,No,490,0,0.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,90,5,0.0,1.0,0.0,2.0,0.0,4.0,0.0,8.0
24989,98,249,499.0,13,0.0,No,499,23,1.0,23,1.0,No,0.782609,0.96,574.548578,9.483146,69,25,5,5,1.0,1.0,2.0,2.0,4.0,4.0,8.0,8.0


In [16]:
data.baskets_train.loc[(data.baskets_train['shopper']==99) & (data.baskets_train['product']==246)]

,week,shopper,product,price,target,category
1488,1,99,246,702,1,13
2260,2,99,246,702,1,13
4581,5,99,246,702,1,13
6125,7,99,246,702,1,13
6903,8,99,246,702,1,13
9161,11,99,246,632,1,13
10658,13,99,246,702,1,13
12167,15,99,246,702,1,13
13694,17,99,246,702,1,13
15236,19,99,246,702,1,13
